In [ ]:
from asim.dataset.scene.scene_builder import ArrowSceneBuilder
from asim.dataset.scene.scene_filter import SceneFilter

In [ ]:
from nuplan.planning.utils.multithreading.worker_sequential import Sequential

# from nuplan.planning.utils.multithreading.worker_ray import RayDistributed

In [ ]:
import os, psutil


def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")


split = "carla"

# log_names = ["2021.06.07.12.54.00_veh-35_01843_02314"]
scene_tokens = None
# scene_tokens = ["875f407079785268"]
log_names = None

scene_filter = SceneFilter(
    split_names=[split], log_names=log_names, scene_tokens=scene_tokens, duration_s=8.1, history_s=1.0
)
scene_builder = ArrowSceneBuilder("/home/daniel/asim_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(len(scenes))

for scene in scenes[:10]:
    print(scene.log_name, scene.token)


# 875f407079785268

# _Rep0_longest1_route0_06_13_17_21_21 457da031d28ba67b
# _Rep0_longest1_route0_06_13_17_21_21 1f6d4a8c9a399b3b
# _Rep0_longest1_route0_06_13_17_21_21 5b7a3e90922db277
# _Rep0_longest1_route0_06_13_17_21_21 827d6ac2ff5356d0
# _Rep0_longest1_route0_06_13_17_21_21 16717aba72175425
# _Rep0_longest1_route0_06_13_17_21_21 5b7444f86f19b444
# _Rep0_longest1_route0_06_13_17_21_21 6ee6004b304f1d3c
# _Rep0_longest1_route0_06_13_17_21_21 c98966ed60a77f7e
# _Rep0_longest1_route0_06_13_17_21_21 790b09ca88553770
# _Rep0_longest1_route0_06_13_17_21_21 b6c1a8d385648623

In [ ]:
from pathlib import Path
from typing import Optional, Tuple

import matplotlib.animation as animation
import matplotlib.pyplot as plt
from tqdm import tqdm

from asim.common.geometry.base import Point2D, StateSE2
from asim.common.geometry.bounding_box.bounding_box import BoundingBoxSE2
from asim.common.visualization.color.default import EGO_VEHICLE_CONFIG
from asim.common.visualization.matplotlib.observation import (
    add_bounding_box_to_ax,
    add_box_detections_to_ax,
    add_default_map_on_ax,
    add_traffic_lights_to_ax,
)
from asim.dataset.arrow.conversion import TrafficLightDetectionWrapper
from asim.dataset.maps.abstract_map import AbstractMap
from asim.dataset.recording.detection.detection import BoxDetectionWrapper
from asim.dataset.scene.abstract_scene import AbstractScene
from nuplan.common.actor_state.ego_state import EgoState
import io
from PIL import Image


def add_ego_vehicle_to_ax_(ax: plt.Axes, ego_state: EgoState) -> None:
    bounding_box = BoundingBoxSE2(
        center=StateSE2(*ego_state.center),
        length=ego_state.car_footprint.length,
        width=ego_state.car_footprint.width,
    )
    add_bounding_box_to_ax(ax, bounding_box, EGO_VEHICLE_CONFIG)


def _plot_scene_on_ax(
    ax: plt.Axes,
    map_api: AbstractMap,
    ego_state: EgoState,
    initial_ego_state: Optional[EgoState],
    box_detections: BoxDetectionWrapper,
    traffic_light_detections: TrafficLightDetectionWrapper,
    radius: float = 120,
) -> plt.Axes:

    if initial_ego_state is not None:
        point_2d = Point2D(initial_ego_state.center.x, initial_ego_state.center.y)
    else:

        point_2d = Point2D(ego_state.center.x, ego_state.center.y)
    add_default_map_on_ax(ax, map_api, point_2d, radius=radius)
    add_traffic_lights_to_ax(ax, traffic_light_detections, map_api)

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax_(ax, ego_state)

    ax.set_xlim(point_2d.x - radius, point_2d.x + radius)
    ax.set_ylim(point_2d.y - radius, point_2d.y + radius)

    ax.set_aspect("equal", adjustable="box")
    return ax


def plot_scene_to_image(
    map_api: AbstractMap,
    ego_state: EgoState,
    initial_ego_state: Optional[EgoState],
    box_detections: BoxDetectionWrapper,
    traffic_light_detections: TrafficLightDetectionWrapper,
    radius: float = 120,
    figsize: Tuple[int, int] = (8, 8),
) -> Image:

    fig, ax = plt.subplots(figsize=figsize)
    _plot_scene_on_ax(ax, map_api, ego_state, initial_ego_state, box_detections, traffic_light_detections, radius)
    ax.set_aspect("equal", adjustable="box")
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    # plt.tight_layout()

    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
from asim.dataset.arrow.conversion import DetectionType
from asim.simulation.gym.demo_gym_env import DemoGymEnv
from asim.simulation.observation.agents_observation import _filter_agents_by_type

import time

images = []
agent_rollouts = []
plot: bool = True
action = [1.0, 0.1]  # Placeholder action, replace with actual action logic
env = DemoGymEnv(scenes)

start = time.time()

map_api, ego_state, detection_observation, current_scene = env.reset(None)
initial_ego_state = ego_state
cars, _, _ = _filter_agents_by_type(detection_observation.box_detections, detection_types=[DetectionType.VEHICLE])
agent_rollouts.append(BoxDetectionWrapper(cars))
if plot:
    images.append(
        plot_scene_to_image(
            map_api,
            ego_state,
            initial_ego_state,
            detection_observation.box_detections,
            detection_observation.traffic_light_detections,
        )
    )


for i in range(150):
    ego_state, detection_observation, end = env.step(action)
    cars, _, _ = _filter_agents_by_type(detection_observation.box_detections, detection_types=[DetectionType.VEHICLE])
    agent_rollouts.append(BoxDetectionWrapper(cars))
    if plot:
        images.append(
            plot_scene_to_image(
                map_api,
                ego_state,
                initial_ego_state,
                detection_observation.box_detections,
                detection_observation.traffic_light_detections,
            )
        )
    if end:
        print("End of scene reached.")
        break

print(time.time() - start)

In [ ]:
import numpy as np


def create_gif(images, output_path, duration=100):
    """
    Create a GIF from a list of PIL images.

    Args:
        images (list): List of PIL.Image objects.
        output_path (str): Path to save the GIF.
        duration (int): Duration between frames in milliseconds.
    """
    if images:
        print(len(images))
        images_p = [img.convert("P", palette=Image.ADAPTIVE) for img in images]
        images_p[0].save(output_path, save_all=True, append_images=images_p[1:], duration=duration, loop=0)


if plot:
    create_gif(images, f"{split}_{0}_{action}.gif", duration=20)

In [ ]:
from typing import List

from asim.simulation.metrics.sim_agents import get_sim_agents_metrics, _get_rollout_agents_array, _get_road_center_distance_feature

from asim.dataset.arrow.conversion import BoxDetection, DetectionType
from asim.dataset.recording.detection.detection import BoxDetectionWrapper



result = get_sim_agents_metrics(current_scene, agent_rollouts)
result





In [ ]:
# from pathlib import Path
# from typing import Optional, Tuple

# import matplotlib.animation as animation
# import matplotlib.pyplot as plt
# from tqdm import tqdm

# from asim.common.geometry.base import Point2D, StateSE2, StateSE2Index
# from asim.common.geometry.bounding_box.bounding_box import BoundingBoxSE2
# from asim.common.geometry.utils import normalize_angle
# from asim.common.visualization.color.default import (
#     BOX_DETECTION_CONFIG,
#     EGO_VEHICLE_CONFIG,
#     MAP_SURFACE_CONFIG,
#     ROUTE_CONFIG,
# )
# from asim.common.visualization.matplotlib.observation import (
#     add_bounding_box_to_ax,
#     add_box_detections_to_ax,
#     add_default_map_on_ax,
#     add_traffic_lights_to_ax,
# )
# from asim.common.visualization.matplotlib.utils import add_shapely_polygon_to_ax
# from asim.dataset.arrow.conversion import TrafficLightDetectionWrapper
# from asim.dataset.maps.abstract_map import AbstractMap
# from asim.dataset.maps.map_datatypes import MapSurfaceType
# from asim.dataset.recording.detection.detection import BoxDetectionWrapper
# from asim.dataset.scene.abstract_scene import AbstractScene
# from nuplan.common.actor_state.ego_state import EgoState
# import io
# from PIL import Image

# from asim.simulation.metrics.map_based import AbstractLane


# # def se2_distance(state1: StateSE2, state2: StateSE2, pos_weight: float = 1.0, heading_weight: float = 1.0):
# #     pos_dist = np.linalg.norm(state1.point_2d.array - state2.point_2d.array)

# #     # Angular distance (shortest path on circle)
# #     dtheta = abs(state1.yaw - state2.yaw)
# #     ang_dist = min(dtheta, 2 * np.pi - dtheta)

# #     # Optional: heavily penalize opposite directions
# #     if ang_dist > np.pi / 2:  # More than 90 degrees apart
# #         ang_dist *= 2  # or return infinity

# #     return pos_weight * pos_dist + heading_weight * ang_dist


# def se2_distance(state1: StateSE2, state2: StateSE2, radius: float):
#     positional_distance = np.linalg.norm(state1.point_2d.array - state2.point_2d.array)
#     abs_yaw_difference = np.abs(normalize_angle(state1.yaw - state2.yaw))
#     rotation_distance = abs_yaw_difference * radius
#     return positional_distance + rotation_distance


# iteration = 0
# map_api = current_scene.map_api
# ego_state = current_scene.get_ego_vehicle_state_at_iteration(iteration)

# rollouts = []
# for i in range(10):
#     rollouts.append(current_scene.get_box_detections_at_iteration(i))
# radius = 80


# fig, ax = plt.subplots(figsize=(8, 8))
# point_2d = Point2D(ego_state.bounding_box.center.x, ego_state.bounding_box.center.y)

# add_default_map_on_ax(ax, map_api, point_2d, radius=radius)
# # add_bounding_box_to_ax(ax, ego_state.bounding_box, EGO_VEHICLE_CONFIG)


# ax.set_xlim(point_2d.x - radius, point_2d.x + radius)
# ax.set_ylim(point_2d.y - radius, point_2d.y + radius)

# ax.set_aspect("equal", adjustable="box")

# def get_agent_tokens(agent_rollout: List[BoxDetection]) -> List[str]:
#     return [
#         box_detection.metadata.track_token
#         for box_detection in agent_rollout
#         if box_detection.metadata.detection_type == DetectionType.VEHICLE
#     ]

# tokens = get_agent_tokens(rollouts[0])
# rollout_agents_array, rollout_agents_mask = _get_rollout_agents_array(rollouts, tokens)


# # mask = rollout_array[:, 0] ==

# distance_feature = _get_road_center_distance_feature(rollout_agents_array, rollout_agents_mask, map_api)
# print(distance_feature.shape)


# token_idx = 34
# token_, iteration_ = tokens[token_idx], 9
# box_detection = rollouts[iteration_].get_detection_by_track_token(token_)

# add_bounding_box_to_ax(ax, box_detection.bounding_box, EGO_VEHICLE_CONFIG)
# print(distance_feature[token_idx, iteration_])

# print(distance_feature.min(), distance_feature[distance_feature!=10].max(), distance_feature.mean(), distance_feature.std())

# np.where(distance_feature==distance_feature[distance_feature!=10].max())
# # distance_feature[distance_feature!=10].max()

In [ ]:
for box in rollouts[0]:
    print(box.metadata.track_token)

In [ ]:
out = map_api.query_nearest(
    point_2d.shapely_point,
    layers=[MapSurfaceType.LANE],
    max_distance=10,
    return_distance=False,
    exclusive=False,
    return_all=True,
)[MapSurfaceType.LANE][0]
print(out)


candidate_lanes = []
for lane_id in out:
    lane = map_api.get_map_object(lane_id, MapSurfaceType.LANE)
    candidate_lanes.append(lane)
    add_shapely_polygon_to_ax(ax, lane.shapely_polygon, ROUTE_CONFIG)


state_se2 = StateSE2(
    ego_state.bounding_box.center.x, ego_state.bounding_box.center.y, ego_state.bounding_box.center.yaw
)

if len(candidate_lanes) == 1:
    current_lane = candidate_lanes[0]
else:
    se2dist = []
    for lane_idx, lane in enumerate(candidate_lanes):
        lane: AbstractLane
        centerline = lane.centerline.polyline_se2
        projected_state_se2 = centerline.interpolate(centerline.project(state_se2.point_2d))
        se2dist.append(se2_distance(state_se2, projected_state_se2, radius=ego_state.bounding_box.length / 2))
    current_lane = candidate_lanes[np.argmin(se2dist)]


add_shapely_polygon_to_ax(ax, current_lane.shapely_polygon, BOX_DETECTION_CONFIG[DetectionType.VEHICLE])

